In [1]:
import tensorflow as tf
import numpy as np
#import pandas as pd
#import cv2
import os
#import tqdm, time
import heapq
import datetime
import glob
import random,time

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from tensorflow.keras.utils import *
from tensorflow.keras.regularizers import * 

#from sklearn.preprocessing import LabelBinarizer
#from sklearn.model_selection import train_test_split
#import matplotlib.pyplot as plt
#import einops

import pathlib
import itertools
#import skvideo.io

from operator import itemgetter 

import random

#from sklearn.metrics import accuracy_score
import random
#from tensorflow.compiler.tf2xla.python import xla

#from tensorflow.keras.constraints import max_norm
#from tqdm.auto import tqdm

2025-01-18 16:21:38.886343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737210098.897844 2757779 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737210098.901494 2757779 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-18 16:21:38.914906: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.layers import RandomFlip, RandomRotation

In [3]:
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [5]:
from delper import*
from delper2 import*

In [7]:
cnxsmall = ConvNeXtSmall()
cnxsmall.load_weights('convnextv1small.h5')

I0000 00:00:1737210100.141510 2757779 service.cc:148] XLA service 0x62e89ed14150 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1737210100.141537 2757779 service.cc:156]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1737210100.273138 2757779 service.cc:148] XLA service 0x62e89c94def0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1737210100.273154 2757779 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
I0000 00:00:1737210100.273157 2757779 service.cc:156]   StreamExecutor device (1): NVIDIA GeForce RTX 4090, Compute Capability 8.9
I0000 00:00:1737210100.282510 2757779 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22279 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:2c:00.0, compute capability: 8.9
I0000 00:00:1737210100.283297 2757779 gpu_device.cc:2022] C

In [8]:
cnxsmall.load_weights('convnextv1small.h5')
mm = cnxsmall

kcv_bone1 = Model(mm.inputs, mm.layers[-2].output)

h = 0
for layer in kcv_bone1.layers[:-1]:
    h=h+1
print(h)

trn = round(h*.7)
for layer in kcv_bone1.layers[:trn]:
 
    layer.trainable = False
    
kcv_bone1.compile(optimizer = keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

kcv_bone1.summary()


333


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 56, 56,    │      4,704 │ input_layer[0][0] │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_ln             │ (None, 56, 56,    │        192 │ stem_conv[0][0]   │
│ (LayerNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack1_block1_dw_p… │ (None, 62, 62,    │          0 │ stem_ln[0][0]     │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack1_block1_dw_c… │ (None, 56, 56,    │      4,800 │ stack1_block1_dw… │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack1_block1_ln    │ (None, 56, 56,    │        192 │ stack1_block1_dw… │
│ (LayerNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack1_block1_up_d… │ (None, 56, 56,    │     37,248 │ stack1_block1_ln… │
│ (Dense)             │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack1_block1_gelu  │ (None, 56, 56,    │          0 │ stack1_block1_up… │
│ (Activation)        │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack1_block1_down… │ (None, 56, 56,    │     36,960 │ stack1_block1_ge… │
│ (Dense)             │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack1_block1_gamma │ (None, 56, 56,    │         96 │ stack1_block1_do… │
│ (ChannelAffine)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack1_block1_outp… │ (None, 56, 56,    │          0 │ stem_ln[0][0],    │
│ (Add)               │ 96)               │            │ stack1_block1_ga… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack1_block2_dw_p… │ (None, 62, 62,    │          0 │ stack1_block1_ou… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack1_block2_dw_c… │ (None, 56, 56,    │      4,800 │ stack1_block2_dw… │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack1_block2_ln    │ (None, 56, 56,    │        192 │ stack1_block2_dw… │
│ (LayerNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack1_block2_up_d… │ (None, 56, 56,    │     37,248 │ stack1_block2_ln… │
│ (Dense)             │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack1_block2_gelu  │ (None, 56, 56,    │          0 │ stack1_block2_up… │
│ (Activation)        │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack1_block2_down… │ (None, 56, 56,    │     36,960 │ stack1_block2_ge

 Total params: 49,454,688 (188.65 MB)

 Trainable params: 25,068,288 (95.63 MB)

 Non-trainable params: 24,386,400 (93.03 MB)

# Split UCF101 folder to train and test splits

In [9]:
!pwd

/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version


In [10]:
from pathlib import Path

In [11]:
data_src = Path.cwd().parent
data_src = data_src / "data/ucf101"
data_src

PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/data/ucf101')

In [12]:
split_list_src = Path.cwd().parent
split_list_src = split_list_src / "data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist"
split_list_src

PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist')

In [13]:
split = 1
train_path = split_list_src / f"trainlist0{split}.txt"
test_path = split_list_src / f"testlist0{split}.txt"
class_info = split_list_src / f"classInd.txt"

In [14]:
lbl2idx = {}
with open(class_info, "r") as f:
    for line in f.readlines():
        idx, lbl = line.split()
        lbl2idx[lbl] = idx

In [15]:
train_paths, train_labels = [],[]
test_paths, test_labels = [],[]

In [16]:
with open(train_path, "r") as f:
    for line in f.readlines():
        path, label = line.split()
        train_paths.append(data_src / path)
        train_labels.append(label)

In [17]:
with open(test_path, "r") as f:
    for line in f.readlines():
        label = line.split("/")[0]
        test_paths.append(data_src / line.strip())
        test_labels.append(lbl2idx[label])

In [18]:
len(train_paths), len(test_paths)

(9537, 3783)

In [19]:
test_paths[:5]

[PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/data/ucf101/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/data/ucf101/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/data/ucf101/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/data/ucf101/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/data/ucf101/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi')]

In [20]:
len(train_labels), len(test_labels)

(9537, 3783)

In [21]:
train_labels[-1]

'101'

In [22]:
tdr1, tdr2 = train_paths, train_labels
vdr1, vdr2 = test_paths, test_labels

In [23]:
vdr1

[PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/data/ucf101/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/data/ucf101/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/data/ucf101/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/data/ucf101/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/data/ucf101/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/data/ucf101/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c06.avi'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/data/ucf101/ApplyEyeMakeup/v_ApplyEyeMakeup_g02_c01.avi'),
 PosixPath('/home/technobothnia/Documents

In [24]:
kcv_f1 = Model(mm.inputs, mm.layers[70].output)
kcv_f2 = Model(mm.inputs, mm.layers[245].output)

In [25]:
class reduce_mean(Layer):
    def call(self, x):
        return tf.reduce_mean(x, [2,3])

In [26]:
class reduce_mean2(Layer):
    def call(self, x):
        return tf.reduce_mean(x, [1,4])

In [27]:
dmv  = 224

input_img = tf.keras.layers.Input(shape=(None,dmv,dmv,3))
 
y0 = tf.keras.layers.TimeDistributed(kcv_bone1)(input_img)

y1 = tf.keras.layers.TimeDistributed(kcv_f1)(input_img)
y1 = reduce_mean()(y1)
y1 = tf.keras.layers.GlobalAveragePooling1D()(y1)



y2 = tf.keras.layers.TimeDistributed(kcv_f2)(input_img)
y2 = reduce_mean2()(y2)
y2 = Flatten()(y2)


y0 = tf.keras.layers.GlobalAveragePooling1D()(y0)


y0 = concatenate([y0,y1,y2],-1)


# y0= Dense(1024 , activation='gelu')(y0)
# y0 = Dropout(0.1)(y0)


# y= Dense(n_cls , activation='softmax')(y0)

mae_bone = Model(input_img, y0)
mae_bone.compile(optimizer = 'adam',
              #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])

mae_bone.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, None, 224, │          0 │ -                 │
│ (InputLayer)        │ 224, 3)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, None, 14,  │  2,751,840 │ input_layer_1[0]… │
│ (TimeDistributed)   │ 14, 384)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_2  │ (None, None, 14,  │ 26,770,656 │ input_layer_1[0]… │
│ (TimeDistributed)   │ 14, 384)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, None, 768) │ 49,454,688 │ input_layer_1[0]… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reduce_mean         │ (None, None, 384) │          0 │ time_distributed… │
│ (reduce_mean)       │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reduce_mean2        │ (None, 14, 14)    │          0 │ time_distributed… │
│ (reduce_mean2)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 768)       │          0 │ time_distributed… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 384)       │          0 │ reduce_mean[0][0] │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 196)       │          0 │ reduce_mean2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 1348)      │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_average_p… │
│                     │                   │            │ flatten[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 49,454,688 (188.65 MB)

 Trainable params: 25,068,288 (95.63 MB)

 Non-trainable params: 24,386,400 (93.03 MB)

In [28]:
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
val_acc_metric   = tf.keras.metrics.CategoricalAccuracy() 


loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
kld = tf.keras.losses.KLDivergence() 

modelo = mae_bone


def msse(y_true, y_pred):
        
    return (tf.keras.losses.mse(y_true, y_pred))


In [29]:
ry = tf.random.normal([2,8,224,224,3])
modelo(ry).shape

/home/technobothnia/.local/lib/python3.10/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(
I0000 00:00:1737199388.286603 2637622 cuda_dnn.cc:529] Loaded cuDNN version 90300


TensorShape([2, 1348])

In [30]:
@tf.function
def train_reg(x1, x2, model1 ):
    
    with tf.GradientTape(persistent = True) as tape:
        
        
        x2 = K.relu(x2)
        tape.watch(x2)
 

        logits1 = model1(x1 , training=True)
        logits2 = model1(x2 , training=True)

        loss_value1 =  msse( logits1, logits2)
        
        
   
    vgrad = tape.gradient(loss_value1, x2)
 
    return vgrad

In [31]:
bzs = 4
cnd = (round(len(tdr1)/bzs)-2)*bzs

tsdr1 = tdr1[0:cnd]
tsdr2 = tdr2[0:cnd]
print(cnd, cnd//bzs)

9576 2394


In [32]:
n_cls = 101

In [33]:
train_ds = B_data_generatoro(tsdr1, tsdr2, bzs, 32, n_cls, shuffle=True, toption= False)

In [34]:
b1 = next(iter(train_ds))[0]

In [35]:
b2 = tf.reduce_mean(b1,[1])
b2 = tf.expand_dims(b2,[1])
b2.shape

TensorShape([4, 1, 224, 224, 3])

In [36]:
def get_frm(trn_st):
    hv1, lv1 = next(iter(trn_st))
    xmw = tf.reduce_mean(hv1,[1])
    oxmw = tf.expand_dims(xmw,[1])
    return oxmw



@tf.function
def train_reg(x1, x2, model1 ):
    
    with tf.GradientTape(persistent = True) as tape:
        
        
        x2 = K.relu(x2)
        tape.watch(x2)
 

        logits1 = model1(x1 , training=True)
        logits2 = model1(x2 , training=True)

        loss_value1 =  msse( logits1, logits2)
        
        
   
    vgrad = tape.gradient(loss_value1, x2)
 
    return vgrad

In [37]:
len(tsdr1) // bzs

2404

In [38]:
len(tdr1)

9624

In [39]:
tsdr2 = [str(int(o) - 1) for o in tsdr2]

In [40]:
tsdr2[-1]

'100'

# CheckMATE distillation

In [ ]:
v_list = list()
l_list = list()

import time

for b in range(0, len(tsdr1) // bzs):
        vd_sub = tsdr1[b * bzs: (b + 1) * bzs]
        lb_sub = tsdr2[b * bzs: (b + 1) * bzs]
        
        train_ds1 = B_data_generatoro(vd_sub, lb_sub, bzs, 64, n_cls, shuffle=False, toption= False)
        #train_ds2 = B_data_generatora2(vd_sub, lb_sub, bzs, 64, n_cls, shuffle=False, toption= False)
        
 
        oxmw =  get_frm(train_ds1)
        #oxmw2 =  get_frm(train_ds2)
        #oxmw = np.concatenate([oxmw1,oxmw2],1)
        
        
        for tk in range(15): #2
        
            train_ds = B_data_generatoro(vd_sub, lb_sub, bzs, 8, n_cls, shuffle=False, toption= False)
            #if tk>5:
            
            if np.random.randint(10)>5:
            
                train_ds = B_data_generatora2(vd_sub, lb_sub, bzs, 8, n_cls, shuffle=False, toption= False)
                
            hv, lv = next(iter(train_ds))
            
            for k in range(3):
            
                xgr = train_reg(hv,oxmw,modelo)
                oxmw = oxmw - xgr*5e-1
                vul = oxmw + xgr*5e-2
                xgv = train_reg(hv,vul,modelo)
                xgr = (xgr+xgv)/2
                
                oxmw = oxmw - xgr*5e-1
     

        print(f"batch {b} distilled")
        v_list.append(oxmw)
        l_list.append(lv)

        # For creating checkpoints while distilling the dataset
        if (b+1)%100 == 0:
            np.save(f"distilled_{b}.npy", np.array(v_list))
            np.save(f"labels_{b}.npy", np.array(l_list))
            v_list, l_list = [],[]
        
np.save(f"distilled_{b}.npy", np.array(v_list))
np.save(f"labels_{b}.npy", np.array(l_list))     
        

E0000 00:00:1735998313.291322 3160652 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape ingradient_tape/functional_3_3/time_distributed_1_1/stack3_block1_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
E0000 00:00:1735998388.399102 3160652 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape ingradient_tape/functional_3_3/time_distributed_1_1/stack3_block1_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


batch 0 distilled
batch 1 distilled
batch 2 distilled


In [21]:
arr = np.load("ucf_split2/distilled_2393.npy")
arr.shape

(94, 4, 1, 224, 224, 3)

In [6]:
arr = np.load("ucf_split2/labels_99.npy")
arr.shape

(100, 4, 101)

In [7]:
arr[0]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
      

In [42]:
src = Path.cwd() / 'ucf_split1'
label_files = [*src.glob("distilled*")]
label_files

[PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version/ucf_split1/distilled_1799.npy'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version/ucf_split1/distilled_99.npy'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version/ucf_split1/distilled_199.npy'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version/ucf_split1/distilled_399.npy'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version/ucf_split1/distilled_899.npy'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version/ucf_split1/distilled_2099.npy'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version/ucf_split1/distilled_999.npy'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version/ucf_split1/distilled_499.npy'),
 PosixPath('/home/technobothnia

In [43]:
import re

In [44]:
def extract_number(filename):
    filename = str(filename.name)
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else 0

In [45]:
extract_number(label_files[0])

1799

In [46]:
sorted_files = sorted(label_files, key=extract_number)
sorted_files

[PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version/ucf_split1/distilled_99.npy'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version/ucf_split1/distilled_199.npy'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version/ucf_split1/distilled_299.npy'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version/ucf_split1/distilled_399.npy'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version/ucf_split1/distilled_499.npy'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version/ucf_split1/distilled_599.npy'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version/ucf_split1/distilled_699.npy'),
 PosixPath('/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version/ucf_split1/distilled_799.npy'),
 PosixPath('/home/technobothnia/D

In [ ]:
!pwd

In [48]:
arr = np.load("ucf_split1/distilled_2381.npy")
arr.shape

(82, 4, 1, 224, 224, 3)

In [49]:
arrays = []
for p in sorted_files:
    arr = np.load(p)
    #arrays.append(arr.reshape(-1, 101)) # for labels
    arrays.append(arr.reshape(-1, 1, 224, 224, 3)) # for distilled videos

In [50]:
arrays[0].shape

(400, 1, 224, 224, 3)

In [51]:
combined = np.concatenate(arrays)

In [52]:
combined.shape

(9528, 1, 224, 224, 3)

In [59]:
combined[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
arr = combined.argmax(axis=1) - 1
arr[0]
arr = tf.keras.utils.to_categorical(arr, num_classes=101)
arr[0]

In [69]:
arr.shape

(9200, 101)

In [41]:
combined = np.load("ucf_split1/labels.npy")
combined.shape

(9200, 101)

In [45]:
arr = np.concatenate((combined, l))
arr.shape


(9528, 101)

In [53]:
#np.save("ucf_split1/distilled.npy", combined)

In [36]:
path = "/home/technobothnia/Documents/leevi_eno/checkmate_fl/simulation/my-app/data"

In [29]:
v_list = np.load(path + "/ucf_split1/distilled.npy")
l_list = np.load(path + "/ucf_split1/labels.npy")
n_cls = 101

In [27]:
v_list = np.array(v_list)
s1,s2,s3,s4,s5,s6 = v_list.shape
vmg = np.reshape(v_list,[s1*s2,s3,s4,s5,s6])
vmg.shape

(10612, 2, 224, 224, 3)

In [26]:
l_list = np.array(l_list)
h1,h2,h3 = l_list.shape
lmg = np.reshape(l_list,[h1*h2,h3]) 
lmg.shape

(10612, 101)

In [24]:
cnxsmall.load_weights('convnextv1small.h5')
mm = cnxsmall

kcv_bone1 = Model(mm.inputs, mm.layers[-2].output)

h = 0
for layer in kcv_bone1.layers[:-1]:
    h=h+1
print(h)

trn = round(h*.9)
for layer in kcv_bone1.layers[:trn]:
 
    layer.trainable = False
    
kcv_bone1.compile(optimizer = keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

333


In [25]:
n_cls = 101
dmv  = 224

input_img = tf.keras.layers.Input(shape=(None,dmv,dmv,3))
y1 = tf.keras.layers.TimeDistributed( RandomFlip("horizontal_and_vertical"))(input_img)  

y2 = tf.keras.layers.TimeDistributed( RandomRotation(0.5))(y1)  

yd = SpatialDropout3D(0.5)(input_img)

y3 = concatenate([input_img,y1,y2,yd],1)

y0 = tf.keras.layers.TimeDistributed(kcv_bone1)(y3)
 
y0 = tf.keras.layers.GlobalAveragePooling1D()(y0)

y0 = Dropout(0.9)(y0)
y0= Dense(1024 , activation='gelu')(y0)
y0 = Dropout(0.5)(y0)


y= Dense(n_cls , activation='softmax')(y0)

cls_bone = Model(input_img, y)
cls_bone.compile(optimizer = 'adam',
              #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              loss = tf.keras.losses.CategoricalCrossentropy(from_logits = False),
              metrics=['accuracy'])

cls_bone.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, None, 224, │          0 │ -                 │
│ (InputLayer)        │ 224, 3)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, None, 224, │          0 │ input_layer_1[0]… │
│ (TimeDistributed)   │ 224, 3)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, None, 224, │          0 │ time_distributed… │
│ (TimeDistributed)   │ 224, 3)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout3d   │ (None, None, 224, │          0 │ input_layer_1[0]… │
│ (SpatialDropout3D)  │ 224, 3)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, None, 224, │          0 │ input_layer_1[0]… │
│ (Concatenate)       │ 224, 3)           │            │ time_distributed… │
│                     │                   │            │ time_distributed… │
│                     │                   │            │ spatial_dropout3… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_2  │ (None, None, 768) │ 49,454,688 │ concatenate[0][0] │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 768)       │          0 │ time_distributed… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 768)       │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1024)      │    787,456 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 1024)      │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 101)       │    103,525 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 50,345,669 (192.05 MB)

 Trainable params: 16,363,493 (62.42 MB)

 Non-trainable params: 33,982,176 (129.63 MB)

In [19]:
cvmg = vmg



print(cvmg.shape, lmg.shape)

(10612, 2, 224, 224, 3) (10612, 101)


cls_bone.load_weights('u91.h5')

In [52]:
v_list = v_list.reshape(-1, 1, 224, 224, 3)

In [32]:
v_list.shape

(9528, 1, 224, 224, 3)

In [33]:
l_list.shape

(9528, 101)

In [34]:
l_list[-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [56]:
indices_arr = l_list.argmax(axis=1) - 1

In [57]:
new_l_list = keras.utils.to_categorical(indices_arr, num_classes=101)

In [58]:
new_l_list[-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [59]:
cls_bone.fit(v_list, new_l_list,16, 20)

2024-11-21 13:10:03.225140: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5539430400 exceeds 10% of free system memory.
2024-11-21 13:10:08.165475: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5539430400 exceeds 10% of free system memory.


Epoch 1/20


E0000 00:00:1732187435.143251 2690146 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/gradient_tape/functional_5_1/time_distributed_5_1/stack4_block2_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


575/575 ━━━━━━━━━━━━━━━━━━━━ 160s 231ms/step - accuracy: 0.1132 - loss: 4.4050
Epoch 2/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 133s 231ms/step - accuracy: 0.6008 - loss: 1.5069
Epoch 3/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 133s 231ms/step - accuracy: 0.7435 - loss: 0.9498
Epoch 4/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 133s 231ms/step - accuracy: 0.8228 - loss: 0.6479
Epoch 5/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 133s 230ms/step - accuracy: 0.8671 - loss: 0.5083
Epoch 6/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 133s 231ms/step - accuracy: 0.8948 - loss: 0.4014
Epoch 7/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 133s 231ms/step - accuracy: 0.9145 - loss: 0.3234
Epoch 8/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 133s 231ms/step - accuracy: 0.9249 - loss: 0.3157
Epoch 9/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 133s 232ms/step - accuracy: 0.9350 - loss: 0.2645
Epoch 10/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 133s 232ms/step - accuracy: 0.9420 - loss: 0.2541
Epoch 11/20
575/575 ━━━━━━━━━━━━━━━━━━━━ 133s 232ms/step - accuracy: 0.9471 - loss: 0.2331
Epoch 12/20
575/575

In [61]:
#cls_bone.save("best.keras")

In [35]:
n_cls = 101

In [34]:
vdr2 = [int(val) - 1 for val in vdr2]

In [41]:
val_ds = B_data_generatoro(vdr1, vdr2, 8, 32, n_cls, shuffle=True, toption= False)

In [28]:
def load_npz_weights(model, npz_path):
    """Load weights from NPZ file into TensorFlow model"""
    weights = np.load(npz_path)
    
    # Convert NPZ file to list of arrays
    weight_list = [weights[key] for key in weights.files]
    
    # Verify and assign weights
    if len(weight_list) != len(model.weights):
        raise ValueError(f"Expected {len(model.weights)} weight arrays, got {len(weight_list)}")
        
    for model_weight, np_weight in zip(model.weights, weight_list):
        if model_weight.shape != np_weight.shape:
            raise ValueError(f"Shape mismatch: {model_weight.shape} vs {np_weight.shape}")
        model_weight.assign(np_weight)

In [29]:
!pwd

/home/technobothnia/Documents/leevi_eno/checkmate_fl/distillation/tf_version


In [30]:
p = "/home/technobothnia/Documents/leevi_eno/checkmate_fl/simulation/my-app"

In [31]:
load_npz_weights(cls_bone, p + "/split1_weights.npz")

In [32]:
len(vdr1)

3783

In [38]:
vdr2[-1], vdr2[0]

(100, 0)

In [47]:
p = Path.cwd().parent.parent / "simulation/my-app/ucf_split1_labels.npy"
train_labels = np.load(p)
train_labels.shape

(9200, 101)

In [50]:
train_labels[-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [48]:
len(vdr1), len(vdr2)

(3734, 3734)

In [37]:
val_acc_metric   = tf.keras.metrics.CategoricalAccuracy() 
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

@tf.function
def test_step(x_batch_val, y_batch_val, model):
    
    val_logits = model(x_batch_val, training=False)
    
    val_loss_value = loss_fn(y_batch_val, val_logits)
    
    val_acc_metric.update_state(y_batch_val, val_logits) 
    
    return val_loss_value

gac = []

for i in range(1):
    gag = []
   
    val_ds = B_data_generatoro(vdr1, vdr2, 2, 40, n_cls, shuffle=True, toption= False)
    for step,  (x_batch_val, y_batch_val) in enumerate( val_ds ): 
 
            
            val_loss_value = test_step(x_batch_val, y_batch_val, cls_bone)
            val_acc = val_acc_metric.result()
            gag.append(val_acc)
        
            #print(step, val_acc.numpy(),  end="\r" ) 
            
            #time.sleep(.001)  
            


    val_acc = val_acc_metric.result()
    gac.append(gag)

print(np.mean(gac),val_acc )

/home/technobothnia/.local/lib/python3.10/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(
I0000 00:00:1737210370.944464 2758015 cuda_dnn.cc:529] Loaded cuDNN version 90300


0.004165362 tf.Tensor(0.0052868095, shape=(), dtype=float32)
